In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
data = pd.read_csv("t20_data.csv.zip", compression="zip")

In [7]:
data.sample(3)

,match_id,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner
56046,1394797,Vanuatu Cricket Ground (Oval 2),Indonesia,field,Indonesia,Vanuatu,2,7,1,16,2.333333,86.0,4.557692,Vanuatu
658408,1451318,"Yeonhui Cricket Ground, Incheon",South Korea,bat,South Korea,China,1,27,7,68,2.219178,NaN,NaN,China
535893,1418182,"Terdthai Cricket Ground, Bangkok",Singapore,bat,Maldives,Singapore,2,5,0,3,5.000000,200.0,10.000000,Singapore


In [8]:
data["batting_team_wins"] = (data["winner"] == data["batting_team"]).astype(int)
data.sample(3)

,match_id,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner,batting_team_wins
770571,1233955,Bay Oval,West Indies,field,West Indies,New Zealand,2,51,2,42,7.116279,239.0,14.461538,New Zealand,0
434064,1284489,"West End Park International Cricket Stadium, Doha",Maldives,bat,Maldives,Saudi Arabia,1,3,0,6,3.000000,NaN,NaN,Saudi Arabia,0
126290,1283042,Gahanga International Cricket Stadium. Rwanda,Botswana,bat,Botswana,Sierra Leone,1,44,1,36,7.135135,NaN,NaN,Botswana,1


In [9]:
# Fill missing numeric with -1
numeric_cols = data.select_dtypes(include=[np.number]).columns
data[numeric_cols] = data[numeric_cols].fillna(-1)

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
# Categorical encoding
categorical_cols = ["venue", "batting_team", "bowling_team", "toss_winner", "toss_decision", "winner"]
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))
    label_encoders[col] = le

In [12]:
data.sample(3)

,match_id,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner,batting_team_wins
577562,1453934,135,50,0,51,32,1,140,4,95,8.571429,-1.0,-1.000000,48,1
219086,1407100,390,67,0,100,68,2,28,1,33,4.666667,100.0,4.965517,94,1
995903,895817,4,1,1,1,41,2,90,3,58,9.000000,189.0,9.580645,40,0


In [13]:
df_inn1 = data[data["innings"] == 1].copy()
df_inn2 = data[data["innings"] == 2].copy()

# Features for innings 1 → drop target-related
features_inn1 = [col for col in df_inn1.columns if col not in [
    "match_id", "winner", "batting_team_wins", 
    "target_runs", "runs_required", "required_rr"
]]

# Features for innings 2 → keep all
features_inn2 = [col for col in df_inn2.columns if col not in [
    "match_id", "winner", "batting_team_wins"
]]

In [14]:
from sklearn.model_selection import GroupShuffleSplit

# Split so same match doesn't leak between train/test
gss1 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx1, test_idx1 = next(gss1.split(df_inn1[features_inn1], df_inn1["batting_team_wins"], groups=df_inn1["match_id"]))

gss2 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx2, test_idx2 = next(gss2.split(df_inn2[features_inn2], df_inn2["batting_team_wins"], groups=df_inn2["match_id"]))

X1_train, X1_test = df_inn1[features_inn1].iloc[train_idx1], df_inn1[features_inn1].iloc[test_idx1]
y1_train, y1_test = df_inn1["batting_team_wins"].iloc[train_idx1], df_inn1["batting_team_wins"].iloc[test_idx1]

X2_train, X2_test = df_inn2[features_inn2].iloc[train_idx2], df_inn2[features_inn2].iloc[test_idx2]
y2_train, y2_test = df_inn2["batting_team_wins"].iloc[train_idx2], df_inn2["batting_team_wins"].iloc[test_idx2]

print("Train size:", X1_train.shape, "Test size:", X1_test.shape)
print("Train size:", X2_train.shape, "Test size:", X2_test.shape)

Train size: (439135, 11) Test size: (109610, 11)
Train size: (374039, 12) Test size: (94765, 12)


In [33]:
from lightgbm import LGBMClassifier

# First innings model
model_inn1 = LGBMClassifier(
    objective="binary",
    learning_rate=0.002,
    num_leaves=31,
    n_estimators=1200,
    random_state=42
)

model_inn1.fit(
    X1_train, y1_train,
    eval_set=[(X1_test, y1_test)],
    eval_metric="auc",
)


[LightGBM] [Info] Number of positive: 218554, number of negative: 220581
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004374 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1195
[LightGBM] [Info] Number of data points in the train set: 439135, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497692 -> initscore=-0.009232
[LightGBM] [Info] Start training from score -0.009232


LGBMClassifier(learning_rate=0.002, n_estimators=1200, objective='binary',
               random_state=42)

In [37]:
# Second innings model
model_inn2 = LGBMClassifier(
    objective="binary",
    learning_rate=0.002,
    num_leaves=31,
    n_estimators=1000,
    random_state=42
)

model_inn2.fit(
    X2_train, y2_train,
    eval_set=[(X2_test, y2_test)],
    eval_metric="auc",
)

[LightGBM] [Info] Number of positive: 169417, number of negative: 204622
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004611 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1687
[LightGBM] [Info] Number of data points in the train set: 374039, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.452939 -> initscore=-0.188801
[LightGBM] [Info] Start training from score -0.188801


LGBMClassifier(learning_rate=0.002, n_estimators=1000, objective='binary',
               random_state=42)

In [38]:
from sklearn.metrics import roc_auc_score, accuracy_score

# First innings
y1_pred_proba = model_inn1.predict_proba(X1_test)[:, 1]
y1_pred = (y1_pred_proba >= 0.5).astype(int)
print("First Innings AUC:", roc_auc_score(y1_test, y1_pred_proba))
print("First Innings Accuracy:", accuracy_score(y1_test, y1_pred))

# Second innings
y2_pred_proba = model_inn2.predict_proba(X2_test)[:, 1]
y2_pred = (y2_pred_proba >= 0.5).astype(int)
print("Second Innings AUC:", roc_auc_score(y2_test, y2_pred_proba))
print("Second Innings Accuracy:", accuracy_score(y2_test, y2_pred))

First Innings AUC: 0.7396501442297218
First Innings Accuracy: 0.6746191040963416
Second Innings AUC: 0.9124063503537123
Second Innings Accuracy: 0.8268981163931831


In [40]:
import pickle

model_bundle = {
    "model_inn1": model_inn1,
    "model_inn2": model_inn2,
    "label_encoders": label_encoders,
    "features_inn1": features_inn1,
    "features_inn2": features_inn2
}

with open("pipe.pkl", "wb") as f:
    pickle.dump(model_bundle, f)

print("✅ Saved two-model setup to pipe.pkl")


✅ Saved two-model setup to pipe.pkl


In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid_logistic = {
    'C': [0.01, 0.1, 1],       # Regularization strength
    'penalty': ['l2'],            # Norm used in the penalization
    'solver': ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
}
inn1_model_logistic = LogisticRegression(max_iter=1000)
inn1_grid_search_logistic = GridSearchCV(inn1_model_logistic, param_grid_logistic, cv=3, scoring='roc_auc', n_jobs=-1)
inn1_grid_search_logistic.fit(X1_train, y1_train)

inn2_model_logistic = LogisticRegression(max_iter=1000)
inn2_grid_search_logistic = GridSearchCV(inn2_model_logistic, param_grid_logistic, cv=3, scoring='roc_auc', n_jobs=-1)
inn2_grid_search_logistic.fit(X2_train, y2_train)

print("Best Parameters (inning 1):", inn1_grid_search_logistic.best_params_)
print("Best Cross-validation Accuracy (inning 1):", inn1_grid_search_logistic.best_score_)

print("Best Parameters (inning 2):", inn2_grid_search_logistic.best_params_)
print("Best Cross-validation Accuracy (inning 2):", inn2_grid_search_logistic.best_score_)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:425: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/optimi

Best Parameters (inning 1): {'C': 0.01, 'penalty': 'l2', 'solver': 'sag'}
Best Cross-validation Accuracy (inning 1): 0.7328448976686911
Best Parameters (inning 2): {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
Best Cross-validation Accuracy (inning 2): 0.9111782693816367


In [ ]:
# First innings
y1_pred_proba_logistic = inn1_grid_search_logistic.predict_proba(X1_test)[:, 1]
y1_pred_logistic = (y1_pred_proba_logistic >= 0.5).astype(int)
print("First Innings AUC:", roc_auc_score(y1_test, y1_pred_proba_logistic))
print("First Innings Accuracy:", accuracy_score(y1_test, y1_pred_logistic))

# Best Parameters (inning 1): {'C': 0.01, 'penalty': 'l2', 'solver': 'sag'}

# Second innings
y2_pred_proba_logistic = inn2_grid_search_logistic.predict_proba(X2_test)[:, 1]
y2_pred_logistic = (y2_pred_proba_logistic >= 0.5).astype(int)
print("Second Innings AUC:", roc_auc_score(y2_test, y2_pred_proba_logistic))
print("Second Innings Accuracy:", accuracy_score(y2_test, y2_pred_logistic))

# Best Parameters (inning 2): {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}

First Innings AUC: 0.7216775825908632
First Innings Accuracy: 0.6606514004196697
Second Innings AUC: 0.9113077265293417
Second Innings Accuracy: 0.8232153221125943


In [53]:
from sklearn.neural_network import MLPClassifier

# Define parameter grid for MLPClassifier
param_grid_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001],       # L2 penalty (regularization term)
    'learning_rate_init': [0.001, 0.01]
}

# Initialize MLP models for inning 1 and inning 2
inn1_model_mlp = MLPClassifier(max_iter=500, random_state=42)
inn2_model_mlp = MLPClassifier(max_iter=500, random_state=42)

# Setup GridSearchCV for both innings
inn1_grid_search_mlp = GridSearchCV(inn1_model_mlp, param_grid_mlp, cv=3, scoring='roc_auc', n_jobs=-1)
inn2_grid_search_mlp = GridSearchCV(inn2_model_mlp, param_grid_mlp, cv=3, scoring='roc_auc', n_jobs=-1)

# Fit grid search on your training data
inn1_grid_search_mlp.fit(X1_train, y1_train)
inn2_grid_search_mlp.fit(X2_train, y2_train)

# Print best parameters and best score
print("Best Parameters (inning 1):", inn1_grid_search_mlp.best_params_)
print("Best Cross-validation ROC AUC (inning 1):", inn1_grid_search_mlp.best_score_)

print("Best Parameters (inning 2):", inn2_grid_search_mlp.best_params_)
print("Best Cross-validation ROC AUC (inning 2):", inn2_grid_search_mlp.best_score_)


Best Parameters (inning 1): {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'learning_rate_init': 0.01, 'solver': 'adam'}
Best Cross-validation ROC AUC (inning 1): 0.728823488100048
Best Parameters (inning 2): {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.01, 'solver': 'adam'}
Best Cross-validation ROC AUC (inning 2): 0.9098525911391612


In [54]:
# First innings
y1_pred_proba_mlp = inn1_grid_search_mlp.predict_proba(X1_test)[:, 1]
y1_pred_mlp = (y1_pred_proba_mlp >= 0.5).astype(int)
print("First Innings AUC:", roc_auc_score(y1_test, y1_pred_proba_mlp))
print("First Innings Accuracy:", accuracy_score(y1_test, y1_pred_mlp))

# Best Parameters (inning 1): {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'learning_rate_init': 0.01, 'solver': 'adam'}  

# Second innings
y2_pred_proba_mlp = inn2_grid_search_mlp.predict_proba(X2_test)[:, 1]
y2_pred_mlp = (y2_pred_proba_mlp >= 0.5).astype(int)
print("Second Innings AUC:", roc_auc_score(y2_test, y2_pred_proba_mlp))
print("Second Innings Accuracy:", accuracy_score(y2_test, y2_pred_mlp))

# Best Parameters (inning 2): {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.01, 'solver': 'adam'}


First Innings AUC: 0.7297250367112332
First Innings Accuracy: 0.6532798102362923
Second Innings AUC: 0.9125942004028986
Second Innings Accuracy: 0.8233630559805836


In [51]:
import xgboost as xgb

param_grid_xgb = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0]
}

inn1_model_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
inn1_grid_search_xgb = GridSearchCV(inn1_model_xgb, param_grid_xgb, cv=3, scoring='roc_auc', n_jobs=-1)
inn1_grid_search_xgb.fit(X1_train, y1_train)

inn2_model_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
inn2_grid_search_xgb = GridSearchCV(inn2_model_xgb, param_grid_xgb, cv=3, scoring='roc_auc', n_jobs=-1)
inn2_grid_search_xgb.fit(X2_train, y2_train)

print("Best Parameters (inning 1):", inn1_grid_search_xgb.best_params_)
print("Best Cross-validation Accuracy (inning 1):", inn1_grid_search_xgb.best_score_)

print("Best Parameters (inning 2):", inn2_grid_search_xgb.best_params_)
print("Best Cross-validation Accuracy (inning 2):", inn2_grid_search_xgb.best_score_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [13:22:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [13:22:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [13:22:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [13:22:04] WARN

Best Parameters (inning 1): {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.7}
Best Cross-validation Accuracy (inning 1): 0.7601113976241507
Best Parameters (inning 2): {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.7}
Best Cross-validation Accuracy (inning 2): 0.9156274799659169


In [52]:
# First innings
y1_pred_proba_xgb = inn1_grid_search_xgb.predict_proba(X1_test)[:, 1]
y1_pred_xgb = (y1_pred_proba_xgb >= 0.5).astype(int)
print("First Innings AUC:", roc_auc_score(y1_test, y1_pred_proba_xgb))
print("First Innings Accuracy:", accuracy_score(y1_test, y1_pred_xgb))

# Best Parameters (inning 1): {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.7}

# Second innings
y2_pred_proba_xgb = inn2_grid_search_xgb.predict_proba(X2_test)[:, 1]
y2_pred_xgb = (y2_pred_proba_xgb >= 0.5).astype(int)
print("Second Innings AUC:", roc_auc_score(y2_test, y2_pred_proba_xgb))
print("Second Innings Accuracy:", accuracy_score(y2_test, y2_pred_xgb))

# Best Parameters (inning 2): {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.7}

First Innings AUC: 0.7623782905767983
First Innings Accuracy: 0.6896086123528875
Second Innings AUC: 0.9144360739700117
Second Innings Accuracy: 0.8263177333403683
